<a href="https://colab.research.google.com/github/peterij/TM10007_Diagnosing_heart_disease/blob/Ingrid/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TM10007 Machine Learning**

In [2]:
!git clone https://github.com/jveenland/tm10007_ml.git

import zipfile
import os
import pandas as pd

with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

Cloning into 'tm10007_ml'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 83 (delta 13), reused 12 (delta 12), pack-reused 64
Unpacking objects: 100% (83/83), 67.93 MiB | 8.86 MiB/s, done.
The number of samples: 827
The number of columns: 9001


In [3]:
# import function
from sklearn import model_selection


In [ ]:
data

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,...,11_741,11_742,11_743,11_744,11_745,11_746,11_747,11_748,11_749,label
0,47.685046,48.416904,37.371002,61.604757,58.732738,213.067966,120.890451,40.100639,19.106303,6.043738,...,0.078753,0.379885,0.182930,0.231552,0.222432,0.154919,0.158616,0.269249,0.112248,0
1,152.726718,297.353726,111.168880,25.419423,14.636085,24.609688,49.533842,253.651571,483.852043,167.617409,...,0.804160,0.809705,0.830032,0.774368,0.798178,0.853519,0.827946,0.859936,0.742673,1
2,1.601260,3.882169,22.978997,21.673040,8.919484,18.329286,27.710604,23.350084,37.896254,8.331180,...,0.156080,0.102070,0.168276,0.120385,0.064254,0.077670,0.059857,0.098506,0.098288,0
3,1.388947,3.052483,3.084103,4.627886,10.016196,15.020347,139.823127,20.149108,10.393109,9.241921,...,0.238274,0.096733,0.030739,0.170214,0.052079,0.179360,0.226222,0.092000,0.192559,0
4,3.625561,3.728466,6.205367,17.722897,5.699401,9.024836,15.782812,33.336480,55.257804,19.139128,...,0.056914,0.059116,0.050098,0.152588,0.129712,0.047342,0.058446,0.141610,0.129477,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,14.673713,13.879148,10.000083,8.337630,3.557302,8.910661,36.647388,5.127375,4.974610,3.649282,...,0.024548,0.056174,0.029438,0.169014,0.111257,0.058975,0.112644,0.155580,0.061931,0
823,3.167367,13.771749,0.705951,0.349087,9.596407,14.721906,48.008495,100.760159,33.764106,19.399167,...,0.808183,0.728553,0.743050,0.735316,0.724321,0.671816,0.714386,0.771741,0.757218,0
824,0.439357,8.700374,20.628752,19.379063,23.176646,25.308323,124.150929,37.162548,13.807592,14.244157,...,0.103234,0.113550,0.102557,0.131152,0.057736,0.061432,0.068036,0.159615,0.077114,0
825,0.235515,4.074581,9.381945,9.621589,16.106126,8.376319,42.678977,40.056677,17.450689,8.489811,...,0.150481,0.073634,0.104637,0.164451,0.055838,0.060012,0.057874,0.060548,0.116350,0


In [7]:
x_train, x_test = model_selection.train_test_split(data, test_size=0.2)

ECG_ABNORMAL = data[data['label'] == 1]
ECG_NORMAL = data[data['label'] == 0]

x_train_abnormal, x_test_abnormal = model_selection.train_test_split(ECG_ABNORMAL, test_size=0.2)
x_train_normal, x_test_normal = model_selection.train_test_split(ECG_NORMAL,test_size = 0.2)

 

print(f'The number of samples: {len(x_train.index)}')
print(f'The number of columns: {len(x_train.columns)}')
# Stratified split met vergelijkbare ratio 


            0_0         0_1         0_2         0_3         0_4         0_5  \
1    152.726718  297.353726  111.168880   25.419423   14.636085   24.609688   
12     6.323226   15.953172   26.120406    5.109627    4.012020   16.225039   
15    18.644972   24.808661   29.530931   34.786282   81.380679   91.139503   
18   247.728859   82.530058   64.737390  101.079961  313.343333  144.387597   
23     7.554942   18.251545   19.131640    4.189196   13.567905   13.140102   
..          ...         ...         ...         ...         ...         ...   
767  883.944360  198.173280   60.302690   15.945958   11.504500    9.191698   
787  875.085192   77.927478   41.706337   21.197205   11.304032    5.111424   
789  823.013130  369.643146  168.177460   57.124367   41.706590   19.712585   
790  278.810394  206.533025   68.386743   68.101193   12.766478  106.676858   
826    0.301328    2.012000    9.122957    7.332689    7.862709   10.139178   

            0_6         0_7         0_8         0_9